# Analysis of the free apps in 2017/2018


As a company we are ad revenue driven. To help with this goal I have worked on
the App Profiles avalible on the 2 major App stores. 

At the end of document I intend to have guidence for the development team on what we should be doing next.

Using free datasets with documentation below:

[Google Play Store dataset](https://www.kaggle.com/lava18/google-play-store-apps/home)

[Apple App Store dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)

In [1]:
from csv import reader

def opencsvlist(datasetloc):
    opened = open(datasetloc, encoding='utf-8')
    read = reader(opened)
    dataset = list(read)
    return dataset


gpsdataset = opencsvlist('googleplaystore.csv')
aasdataset = opencsvlist('AppleStore.csv')

print(gpsdataset[:3])
print('------------------------------------------------------------------')
print(aasdataset[:3])


[['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'], ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up'], ['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']]
------------------------------------------------------------------
[['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic'], ['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1'], ['389801252', 'Instagram', '113954816', 'USD

# Useful variables required repeatedly

The first chunk of code is to check if a known bad row exists and the delete it if required. It also prints pre and post the check for visual confirmation we are not damaging other rows

- `Lenghtcheck` are for making lenghtcheck's easier to read, consistant and typing it was getting annoying

- `X_app_name` is the array location for the name in a particular dataset
- `X_app_review` is the same for the reviews
- `X_app_price` is the location of pricing data
- `X_app_genre` is the location for genre data
- `google_app_installs`is the locations for the number of installs, please note we do not have this data for the Apple App Store

In [2]:
#This block checks for a known bad row and removes it 10473
print('Bad row check :')
print('\n')
print(gpsdataset[10473])
if gpsdataset[10473][0] == 'Life Made WI-Fi Touchscreen Photo Frame':
    del gpsdataset[10473]    
print('\n')
print(gpsdataset[10473])
print('------------------------------------------------------------------------')

def lenghtcheck(dataset_google,dataset_apple):
    print('\n')
    print('Results from this code block are:')
    print('\n')
    print('The lenght of this function Google dataset is now : ', len(dataset_google))
    print('\n')
    print('The lenght of this function Apple dataset is now : ', len(dataset_apple))
    print('\n')
    print('----------------------------------------------------------------------')
    

google_app_name = 0 #Array location for name's of applications - Google
apple_app_name = 1 #Array location for name's of applications - Apple

google_app_review = 3 #Array location for reviews - Google
apple_app_review = 5 #Array location for reviews - Apple

google_app_price = 7 #Array location for prices - Google
apple_app_price = 4 #Array location for prices - Apple

google_app_genre = 9 #Array location for genres - Google
google_app_category = 1#Array location for category - Google
apple_app_genre = 11 #Array location for primary genre - Apple

google_app_install = 5 #Array Location for number of installs



Bad row check :


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']
------------------------------------------------------------------------


## Checking for duplicates and number of duplicates

The below function is for checking for duplicate app names.

- The function can return either the unique or the duplicate application names
- It needs to be called with at least one varibale which is the `dataset` you wish it to work on
- Returns a list, by default it returns the duplicate names. 

We will later remove the duplicates by selecting for the highest number of reviews as this is likely fresher data.

In [3]:
def duplicates(dataset,arraylocname,duplicates=True):
    duplicate_apps = []
    unique_apps = []
    for app in dataset:
        name = app[arraylocname]
        if name in unique_apps:
            duplicate_apps.append(name)
        else:
            unique_apps.append(name)
    if duplicates:
        return duplicate_apps
    return unique_apps

gpsdatasetdups = duplicates(gpsdataset,google_app_name)
aasdatasetdups = duplicates(aasdataset,apple_app_name)

print(gpsdatasetdups[:10])
print('\n')
print('Number of duplicate apps: ', len(gpsdatasetdups))
print('Number of unique apps: ', (len(duplicates(gpsdataset,google_app_name,False)) -1))
print('-----------------------------------------------------------------------')
print(aasdatasetdups[:10])
print('\n')
print('Number of duplicate apps: ', len(aasdatasetdups))
print('Number of unique apps: ', (len(duplicates(aasdataset,apple_app_name,False)) -1))

['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack']


Number of duplicate apps:  1181
Number of unique apps:  9659
-----------------------------------------------------------------------
['Mannequin Challenge', 'VR Roller Coaster']


Number of duplicate apps:  2
Number of unique apps:  7195


## Doing something about the duplicates

First we are going to get the highest reviewes for all the duplicate (and unique) apps

We are going to do this by taking the `App` name and the number of reviews into a seperate `dictionary`
- This is going to be achieved by a `for` loop and 2 `if` statements

While we currently only have to do the Google Play Store dataset I am going to write this as a function for later use if required
- This is take a single arguement which will be the dataset required to be worked on
- It will return a dictionary with the app name being the key and reviews being the value

In [4]:
def highestreviewsdict(dataset,arraylocname,arraylocreview):
    reviews_max = {}
    for app in dataset[1:]:
        name = app[arraylocname]
        n_reviews = float(app[arraylocreview])
        if name in reviews_max and reviews_max[name] < n_reviews:
            reviews_max[name] = n_reviews
        elif name not in reviews_max:
            reviews_max[name] = n_reviews
    return reviews_max

gpsmaxreviews = highestreviewsdict(gpsdataset,google_app_name,google_app_review)
aasmaxreviews = highestreviewsdict(aasdataset,apple_app_name,apple_app_review)

lenghtcheck(gpsmaxreviews,aasmaxreviews)



Results from this code block are:


The lenght of this function Google dataset is now :  9659


The lenght of this function Apple dataset is now :  7195


----------------------------------------------------------------------


### Using the dictionary we just created

Now we are going to clean the list of duplicates using the dictionary of unique names and highest number of reviews.

We are going to do this with another `loop` and set of `if` statements

And again I am going to create this as a function
    - Please note that this function removes the header

In [5]:
def dedupwithreviews(dataset,reviews,arraylocname,arraylocreview):
    cleaned = []
    already_added = []
    for row in dataset[1:]:
        name = row[arraylocname]
        n_reviews = float(row[arraylocreview])
        if reviews[name] == n_reviews and name not in already_added:
            cleaned.append(row)
            already_added.append(name)
    return cleaned

gpsdeduped = dedupwithreviews(gpsdataset,gpsmaxreviews,0,3)
aasdeduped = dedupwithreviews(aasdataset,aasmaxreviews,1,5)

lenghtcheck(gpsdeduped,aasdeduped)    



Results from this code block are:


The lenght of this function Google dataset is now :  9659


The lenght of this function Apple dataset is now :  7195


----------------------------------------------------------------------


## Targeting an English Audience

- Using ASCII numbering we are going to check if a member of the string is over 127 which is where the English character set lies
- We are going to loop through the individual characters of the names
- Due to items like emoji's and copywrite marks we will require 3 none English characters

The reason we are doing the above is that if a name contains more than 3 non English characters then it is likely targeted at a non English speaking audience. 

In [6]:
def asciicheck127(name):
    letter = ''
    count = 0
    for character in name:
        if ord(character) >= 127:
            count += 1
        elif count == 3:
            return False
    return True

test = asciicheck127('PPS奇奇PPS')
print(test)

True


### Now to use the function `asciicheck127` to clean our data

We are going to creat a function to clean the 2 datasets of apps we are assuming are targeting non English speaking audiences

It does a for loop through the dataset extracting the name and passing that to `asciicheck127`  
- Using the returned boolean to decide weather or not to add the list to the `englishonly` list of lists

In [7]:
def nonenglishclean(dataset,arraynameloc):
    englishonly = []
    name = ''
    for row in dataset:
        name = row[arraynameloc]
        if asciicheck127(name):
            englishonly.append(row)
    return englishonly

gpsdedupenglish = nonenglishclean(gpsdeduped,google_app_name)
aasdedupenglish = nonenglishclean(aasdeduped,apple_app_name)

lenghtcheck(gpsdedupenglish,aasdedupenglish)



Results from this code block are:


The lenght of this function Google dataset is now :  9636


The lenght of this function Apple dataset is now :  7114


----------------------------------------------------------------------


## Removal of paid apps 

Being that we as a company do not charge for any of our apps we need to remove the app that are charged for

Using a similar function as we have been using throughout we are going to loop through the apps checking on price and returning only those that are free to the end user.
- The function will be built to return free or paid apps but will default to free
   - Writen in such a way that the loop is only done for the selected type of app

In [8]:
def chargeable_checker(dataset,arraypriceloc,Free=True):
    freeapps = []
    paidapps = []
    app_price = 0.0
    if Free:
        for row in dataset:
            app_price = row[arraypriceloc]
            if app_price == '0' or app_price == '0.0':
                freeapps.append(row)
        else:
            for row in dataset:
                app_price = row[arraypriceloc]
                if app_price is not '0' or app_price is not '0.0':
                    paidapps.append(row)
        
    if Free:
        return freeapps
    return paidapps

gpsdedupengfree = chargeable_checker(gpsdedupenglish,google_app_price)
aasdedupengfree = chargeable_checker(aasdedupenglish,apple_app_price)

lenghtcheck(gpsdedupengfree,aasdedupengfree)



Results from this code block are:


The lenght of this function Google dataset is now :  8884


The lenght of this function Apple dataset is now :  3983


----------------------------------------------------------------------


## App profiles

We need to make the best use of our resources so we need to find something that works for the Google Play Store as well as the Apple App Store. As the Google Play Store has less stringent entry process we will first target that
    - As per our normal opterating proceedure we will build a MVP for release
    - If it gains traction with our user profile then we can develop it further
    - If profitable after 6 months we will create the IOS version 

A big difference in the datasets is that the Google set includes 2 fields and multiple genres where as the Apple set only includes the primary genre

I am going to split this into 2 functions, one to display the frequency table and one to produce the table.
    
- `freq_table` Will product the tables
    - It will require which dataset (list of lists) to work on and the index (int) you require a frequency table of.
    - It will return a dictionary with the keys as items in the index and values as percentages of the dataset
    

- `display_table` Will turn the tables into something easier to read.
    - It will require which dataset (list of lists) to work on and the index (int) you require a frequency table displayed
    - It will call `freq_table` passing both arguements
    - It will loop through the returned dictionary turning it into a tuple with the key as the dictionary's value and the value as the dictionary's key
        - For example  "games : 99" in the dictionary becomes "99 : games" in the tuple
    - It will sort the tuples into number order highest to lowest using `sorted(x, reverse = True)` and convert this into a list of lists
    - This will then print the list but in the dictionary order so genre is first not second
        - For example "99 : games" will be printed "games : 99"

In [9]:
def freq_table(dataset,index):
    freq = {}
    genre = ''
    for row in dataset:
        genre = row[index]
        if genre in freq:
            freq[genre] += 1
        else:
            freq[genre] = 1
    for key in freq:
        freq[key] = round((freq[key] / len(dataset)) * 100,1)
    return freq

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

print("-" * 50)
print("\n")
apple_table = display_table( aasdedupengfree, apple_app_genre)
print("\n")
print("-" * 50)
print("\n")
google_cat_table = display_table( gpsdedupengfree, google_app_category)
print("\n")
print("-" * 50)
print("\n")
google_genre_table = display_table( gpsdedupengfree, google_app_genre)
print("\n")
print("-" * 50)

--------------------------------------------------


Games : 56.2
Entertainment : 8.2
Photo & Video : 4.2
Social Networking : 3.6
Education : 3.2
Shopping : 2.8
Utilities : 2.6
Lifestyle : 2.2
Sports : 1.9
Health & Fitness : 1.9
Finance : 1.9
Music : 1.7
Productivity : 1.6
News : 1.5
Book : 1.5
Travel : 1.4
Food & Drink : 1.1
Weather : 0.8
Reference : 0.5
Navigation : 0.5
Business : 0.5
Medical : 0.2
Catalogs : 0.2


--------------------------------------------------


FAMILY : 19.0
GAME : 9.7
TOOLS : 8.4
BUSINESS : 4.6
PRODUCTIVITY : 3.9
LIFESTYLE : 3.9
FINANCE : 3.7
MEDICAL : 3.5
SPORTS : 3.4
PERSONALIZATION : 3.3
COMMUNICATION : 3.2
HEALTH_AND_FITNESS : 3.1
PHOTOGRAPHY : 2.9
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.7
TRAVEL_AND_LOCAL : 2.3
SHOPPING : 2.3
BOOKS_AND_REFERENCE : 2.2
DATING : 1.9
VIDEO_PLAYERS : 1.8
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.2
EDUCATION : 1.2
ENTERTAINMENT : 1.0
LIBRARIES_AND_DEMO : 0.9
AUTO_AND_VEHICLES : 0.9
WEATHER : 0.8
HOUSE_AND_HOME : 0.8
PARENTING :

## User ratings/Installs

We are now going to include the user ratings into our analysis for the apple set and the number of installs for the google set. Due to the mismatch of the data we received we have used the closest possible metrics. 

Due to the google install data being a string containing `+` and `,` we will have to do some more data cleaning. 

In [10]:
apple_genre_freq_table = freq_table( aasdedupengfree, apple_app_genre)
google_category_freq_table = freq_table( gpsdedupengfree, google_app_category)
google_genre_freq_table = freq_table( gpsdedupengfree, google_app_genre)

def user_rated_genres( dataset, freqtableset, arraylocgenre, arraylocinstalls):
    genre_avg_rating = []
    for genre in freqtableset:
        total = 0
        len_genre = 0
        avg_rating = 0
        for row in dataset:
            genre_app = row[arraylocgenre]
            if genre == genre_app:
                ratings = row[arraylocinstalls]
                ratings = ratings.replace('+','')
                ratings = ratings.replace(',','')
                total += float(ratings)
                len_genre += 1
        avg_rating = round(total / len_genre)
        genre_avg_rating.append([avg_rating,genre])
    genre_avg_rating = sorted(genre_avg_rating,reverse = True)
    return genre_avg_rating

apple_genres_reviews = user_rated_genres( aasdedupengfree, apple_genre_freq_table, apple_app_genre, apple_app_review)
google_category_reviews = user_rated_genres( gpsdedupengfree, google_category_freq_table, google_app_category, google_app_install)
google_genres_reviews = user_rated_genres( gpsdedupengfree, google_genre_freq_table, google_app_genre, google_app_install)

print(apple_genres_reviews[:5])
print(google_category_reviews[:5])
print(google_genres_reviews[:5])

[[70998, 'Reference'], [56482, 'Music'], [53452, 'Social Networking'], [48795, 'Weather'], [27414, 'Photo & Video']]
[[38456154, 'COMMUNICATION'], [24727872, 'VIDEO_PLAYERS'], [23253652, 'SOCIAL'], [17772019, 'PHOTOGRAPHY'], [16738958, 'PRODUCTIVITY']]
[[38456154, 'Communication'], [35333333, 'Adventure;Action & Adventure'], [24947336, 'Video Players & Editors'], [23253652, 'Social'], [23028171, 'Arcade']]


# Conclusion

The consistant between all 3 top 5 outputs are:

Social Networking/ Social

and 

Photo/Video playing and editing

For our small team I feel we might have more success at a Photo/Video application than creating the next facebook. 